In [1]:
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import dictionary_corpus
from dictionary_corpus import Corpus
import numpy as np
from sklearn.decomposition import PCA, SparsePCA, KernelPCA, IncrementalPCA
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
import pickle as pk
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 30)

matplotlib.use('webagg')
# load the model and the corpus
model = torch.load('hidden650_batch128_dropout0.2_lr20.0.pt',map_location=torch.device('cpu'))
corpus = Corpus('')
# print("Vocab size %d", ntokens)
import matplotlib.rcsetup as rcsetup
print(rcsetup.all_backends)

['GTK3Agg', 'GTK3Cairo', 'GTK4Agg', 'GTK4Cairo', 'MacOSX', 'nbAgg', 'QtAgg', 'QtCairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']


/home/ezio/.cache/pypoetry/virtualenvs/polysemy-EBh2Gjjb-py3.7/lib/python3.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'model.RNNModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ezio/.cache/pypoetry/virtualenvs/polysemy-EBh2Gjjb-py3.7/lib/python3.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ezio/.cache/pypoetry/virtualenvs/polysemy-EBh2Gjjb-py3.7/lib/python3.7/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'to

In [2]:
def euclidean(x,y):
    dist = np.linalg.norm(x-y)
    return dist.round(2)
def cos(x,y):
    dist = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    return dist.round(2)
def find_top_k_sim(dfk,k,tar):
    l = []
    v = dfk.loc[tar]['tensors']
    dfk['dist']= dfk['tensors'].apply(lambda x: euclidean(x,v))
    l = sorted(list(dfk['dist']))[::k]
    return dfk.sort_values(by='dist',ascending=True)[:k]

In [3]:
with open('all_word_h', 'rb') as f:
     y_ori = pk.load(f).set_index('labels')
with open('all_sent_h1', 'rb') as f:
    x1 = pk.load(f)
x1['num'] = x1.index
x1['target']= x1['labels'].apply(lambda x: x.split(' ')[-1][:-1])
x1['file']= x1['labels'].apply(lambda x: x.split(' ')[-1][-1])
x1['num']= x1['num'].apply(lambda x: str(x))
x1['labels'] = x1['num']+x1['labels']

x1=x1.drop(['num'],axis=1).set_index('labels')
all_df = {}
for each in x1.groupby(by='target'):
    y_ori = pd.concat([y_ori,each[1].sort_values(by='file').drop(['target','file'],axis=1)])
    all_df[each[0]]=each[1]
len(y_ori)

51043

In [86]:
tar = 'books'

In [87]:
# find top similairty for a list of words
d = {}
for each in tqdm([word for word in all_df[tar].index]):
    a = find_top_k_sim(y_ori,10,each)
    a['s'] = a['dist'].apply(lambda x: str(x)[:4])
    a['s'] = a.index+' '+a['s']
    d[each] =  list(a['s'])
dfmat = pd.DataFrame.from_dict(d)
dfmat

100%|███████████████████████████████████████████| 41/41 [00:33<00:00,  1.21it/s]


,305favorite books2,306hundred books2,307with books2,308important books2,309good books2,310of books2,311log books2,312their books2,313electronic books2,314of books2,315comic books2,316particular books2,317publish books2,318balance books2,319includes books2,...,804and books1,805it books1,806she books1,807which books1,808it books1,809agent books1,810who books1,811that books1,812who books1,813and books1,814car books1,815never books1,816party books1,817who books1,818she books1
0,305favorite books2 0.0,306hundred books2 0.0,307with books2 0.0,308important books2 0.0,309good books2 0.0,310of books2 0.0,311log books2 0.0,312their books2 0.0,313electronic books2 0.0,314of books2 0.0,315comic books2 0.0,316particular books2 0.0,317publish books2 0.0,318balance books2 0.0,319includes books2 0.0,...,804and books1 0.0,805it books1 0.0,806she books1 0.0,807which books1 0.0,808it books1 0.0,809agent books1 0.0,810who books1 0.0,811that books1 0.0,812who books1 0.0,813and books1 0.0,814car books1 0.0,815never books1 0.0,816party books1 0.0,817who books1 0.0,818she books1 0.0
1,308important books2 2.95,308important books2 3.14,books 2.97,324poetry books2 2.75,ourselves 3.29,books 3.22,316particular books2 3.10,317publish books2 3.49,317publish books2 3.01,books 3.13,313electronic books2 3.34,311log books2 3.10,books 2.70,319includes books2 3.17,books 2.69,...,books 3.05,802normally books1 2.99,812who books1 2.56,799who books1 2.98,176to book1 2.68,799who books1 3.33,812who books1 2.58,812who books1 2.63,799who books1 2.13,810who books1 3.52,379own moves2 2.78,812who books1 3.03,812who books1 2.94,812who books1 2.54,806she books1 2.89
2,books 2.99,books 3.40,publications 3.19,books 2.78,ones 3.31,papers 3.29,registers 3.11,books 3.51,products 3.08,papers 3.24,320and books2 3.52,books 3.14,319includes books2 2.88,books 3.20,317publish books2 2.88,...,releases 3.21,798who books1 3.05,799who books1 2.67,802normally books1 3.23,801she books1 2.72,807which books1 3.35,799who books1 2.80,799who books1 2.89,817who books1 2.54,804and books1 3.74,808it books1 3.10,806she books1 3.05,808it books1 3.04,799who books1 2.83,812who books1 3.02
3,novels 3.14,305favorite books2 3.49,texts 3.19,poems 2.92,myself 3.33,319includes books2 3.36,texts 3.17,publications 3.57,books 3.09,texts 3.31,books 3.58,publications 3.22,publications 2.90,booklets 3.30,articles 2.96,...,papers 3.23,808it books1 3.10,818she books1 2.89,805it books1 3.29,184to book1 2.80,812who books1 3.55,818she books1 3.07,798who books1 2.90,806she books1 2.56,799who books1 3.75,stuff 3.13,799who books1 3.27,801she books1 3.15,798who books1 2.90,810who books1 3.07
4,poems 3.15,poems 3.51,poems 3.25,305favorite books2 2.95,them 3.34,poems 3.39,cards 3.17,313electronic books2 3.59,computers 3.23,320and books2 3.34,321antique books2 3.71,novels 3.26,texts 2.96,publications 3.36,papers 2.97,...,purchases 3.25,799who books1 3.11,815never books1 3.05,800he books1 3.32,190also book1 2.82,816party books1 3.59,808it books1 3.11,817who books1 2.98,810who books1 2.58,807which books1 3.89,801she books1 3.22,798who books1 3.32,177you book1 3.25,811that books1 2.98,799who books1 3.08
5,publications 3.20,papers 3.53,writings 3.25,writings 2.98,stuff 3.34,textbooks 3.41,books 3.18,them 3.60,toys 3.23,files 3.38,magazines 3.72,poems 3.26,papers 2.99,papers 3.39,texts 2.99,...,catalogues 3.26,812who books1 3.25,798who books1 3.16,809agent books1 3.35,177you book1 2.83,810who books1 3.66,190also book1 3.22,802normally books1 3.29,811that books1 2.63,812who books1 3.95,ones 3.22,176to book1 3.32,805it books1 3.28,810who books1 3.41,800he books1 3.42
6,324poetry books2 3.22,writings 3.63,stories 3.27,papers 2.98,themselves 3.35,texts 3.44,files 3.20,papers 3.63,objects 3.24,pamphlets 3.38,pens 3.73,texts 3.32,313electronic books2 3.01,poems 3.39,poems 3.00,...,packs 3.26,816party books1 3.28,801she books1 3.17,812who books1 3.38,178can book1 2.90,806she books1 3.68,178can book1 3.25,816party books1 3.32,798who books1